# 1. 허깅페이스를 통한 한국어 말뭉치 데이터가 학습된, Llama 3.1 8b를 가져오기. (from :Bllossom)

In [1]:
from vllm import LLM, SamplingParams
import torch
from transformers import AutoTokenizer

/home/user/miniconda3/envs/vllm_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-04 10:24:54,124	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
device = 'cuda' if torch.cuda.is_available() else "cpu"  #gpu 확인을 위해 사용
print(device)
print(torch.__version__)  # PyTorch 버전 확인
print(torch.version.cuda)  # PyTorch가 사용하는 CUDA 버전 확인
print(torch.cuda.is_available())  

cuda
2.5.1+cu124
12.4
True


In [3]:
model_name = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
#model_name="facebook/opt-125m" 
llm = LLM(model=model_name, dtype="float16", gpu_memory_utilization=0.95,max_model_len=3072, tokenizer=model_name, tokenizer_mode='auto')
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# ### https://huggingface.co/Bllossom/llama-3.2-Korean-Bllossom-3B 참조.
# model=  AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map="cuda",
# )

# model.device

INFO 03-04 10:25:00 __init__.py:207] Automatically detected platform cuda.
WARNING 03-04 10:25:01 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-04 10:25:06 config.py:549] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
INFO 03-04 10:25:06 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Bllossom/llama-3.2-Korean-Bllossom-3B', speculative_config=None, tokenizer='Bllossom/llama-3.2-Korean-Bllossom-3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=3072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_confi

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:02<00:02,  2.06s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.37s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.47s/it]


INFO 03-04 10:25:12 model_runner.py:1115] Loading model weights took 6.0160 GB


INFO 03-04 10:25:14 worker.py:267] Memory profiling takes 1.97 seconds
INFO 03-04 10:25:14 worker.py:267] the current vLLM instance can use total_gpu_memory (8.00GiB) x gpu_memory_utilization (0.95) = 7.60GiB
INFO 03-04 10:25:14 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 0.36GiB.
INFO 03-04 10:25:14 executor_base.py:111] # cuda blocks: 209, # CPU blocks: 2340
INFO 03-04 10:25:14 executor_base.py:116] Maximum concurrency for 3072 tokens per request: 1.09x
INFO 03-04 10:25:15 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as need

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.33it/s]

INFO 03-04 10:25:30 model_runner.py:1562] Graph capturing finished in 15 secs, took 0.16 GiB
INFO 03-04 10:25:30 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 18.44 seconds


In [ ]:
article ="""
블랙핑크 멤버들의 솔로 작품이 연달아 좋은 반응을 얻고 있다. 블랙핑크는 2023년 YG엔터테인먼트와 팀 활동 재계약을 맺으면서 개인 활동은 각자 새로운 둥지를 찾거나 꾸려서 하기로 했다. 이후 리사, 로제, 제니가 선보인 작품은 하나같이 세계 정상급 자본력과 기획력이 결합해야 나올 수 있는 퀄리티로 화제를 모았다. 올해 블랙핑크 컴백이 예상되는 가운데 지수가 2월 14일 미니 앨범 ‘아모르타주(AMORTAGE)’를 발매함으로써 네 멤버 모두 각자의 작업물을 내놓게 됐다.
‌타이틀곡 ‘어스퀘이크(earthquake)’는 듣다 보면 어쩐지 교묘하게 뒤엉킨 듯한 기분이 든다. 노래 각 부분이 서로 다른 방식으로 선명하게 인상을 남기기 때문이다. 호흡을 짚어가며 단음 위주로 흐르던 보컬이 “벗어나려 할수록…”으로 진입할 때면 갑자기 멜로디컬해지고, 심지어 1990년대 댄스가요 같은 인상마저 준다. “바우트 투 블로(Bout to blow)”라는 가사 그대로 터질 것만 같던 긴장이 시원하게 해소되는 건 후렴 부분부터다.
블랙핑크의 또 다른 세계
간단하되 오묘한 모티프로 음역을 옮겨가며 반복되는 후렴은 기하학적인 인상을 준다. 뜨거운 훅은 아니지만 선명하게 각인될 만하고, 무엇보다 안정감이 빼어나다. ‘이제 노래가 제자리로 돌아왔다’는 기분을 느끼게 한다. 거기에 시원하게 달려 나가는 클럽 스타일 하우스 비트가 결합한다. 기분은 ‘이제 춤출 일만 남았다’로 바뀐다. 마침 뮤직비디오 속 지수도 댄서들을 ‘거느리고’ 춤을 춘다. 이는 분명 팝 시장에서 하우스 음악이 제공할 수 있는 가장 좋은 점 가운데 하나다.
‌뮤직비디오는 K팝의 정수를 가져와 멋들어지게 표현해낸다. 전체주의적인 사회에서 권력과 감성이 대결한다는 설정, 질주하는 슈퍼카, 화려한 군무 같은 것은 K팝을 대외적으로 대표하는 이미지라 할 만하다. 어둡고 위압적인 사무실 풍경이나 거대한 스마트폰 자판 위에서 춤추는 댄서들 모습 등을 보면 금붙이 하나 없이도 ‘부티’가 흐른다는 느낌이 든다. 그리고 “그를 믿어도 될까”라는 질문 앞에서 가소롭다는 듯한 웃음을 띠며 가속페달을 밟아버리는 지수의 모습이 전하는 쾌감도 보통이 아니다.
‌이미 개인 작품을 선보인 다른 멤버들에 비해 지수의 노선은 좀 더 K팝적이다. K팝 정상에서 만들어낼 수 있는 것, 그러나 역시 블랙핑크 틀 안에서는 도무지 상상하기 어려운 어떤 것이다. 네 멤버의 네 방향 이정표가 이렇게 완성됐다. 이를 따라 확장될 새로운 블랙핑크의 세계를 기대하지 않기 어렵다.
"""

keyword ="블랙핑크"

prompt = f"""
요약 규칙:
1. {keyword}가 포함된 내용 위주로 요약해주세요.
2. {keyword}가 포함되지 않은 부분은 생략해주세요.
3. 5문장 이상 10문장 이하로 요약해 주세요.
4. 같은 내용의 문장을 반복하지 마세요.
5. 모르는 부분은 대답하지 마세요.
6. 문장을 생성할 때 구어체로 이어지게 해주세요.

다음 문장을 요약해주세요.: \n
{article} \n

요약:
<|eot_id|>
"""


sampling_params = SamplingParams(
    temperature=0.4,
    top_p=0.7,
    max_tokens=512
)

# 텍스트 생성
outputs = llm.generate(prompt, sampling_params=sampling_params)
print(outputs[0].outputs[0].text)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.62s/it, est. speed input: 59.22 toks/s, output: 32.78 toks/s]

블랙핑크 멤버들이 각각의 솔로 작품을 통해 좋은 반응을 얻고 있다. 블랙핑크는 YG엔터테인먼트와 팀 활동 재계약을 맺고, 개인 활동은 각자 새로운 둥지를 찾거나 꾸려서 하기로 했다. 리사, 로제, 제니도 각각의 작품이 화제를 모았다. 지수는 2월 14일 ‘아모르타주(AMORTAGE)’를 발매하며 네 멤버 모두 각자의 작업물을 내놓게 됐다. 타이틀곡 ‘어스퀘이크(earthquake)’는 보컬이 단음 위주로 흐르며, 후렴 부분은 기하학적인 인상을 준다. 뮤직비디오는 K팝의 정수를 가져와 멋들어지게 표현해낸다. 지수의 노선은 K팝적이고, 블랙핑크 틀 안에서는 도무지 상상하기 어려운 것이었다. 네 멤버의 네 방향 이정표가 완성됐다. 이를 따라 확장될 새로운 블랙핑크의 세계를 기대하지 않기 어렵다. 블랙핑크의 또 다른 세계를 기대해 볼 수 있다. 

블랙핑크의 새로운 세계를 기대해 볼 수 있다. 지수의 솔로 작품이 K팝적이고, 블랙핑크 틀 안에서는 도무지 상상하기 어려운 것이었다. 네 멤버의 네 방향 이정표가 완성됐다. 이를 따라 확장될 새로운 블랙핑크의 세계를 기대하지 않기 어렵다. 블랙핑크의 또 다른 세계를 기대해 볼 수 있다. 지수의 솔로 작품이 K팝적이고, 블랙핑크 틀 안에서는 도무지 상상하기 어려운 것이었다. 네 멤버의 네 방향 이정표가 완성됐다. 이를 따라 확장될 새로운 블랙핑크의 세계를 기대하지 않기 어렵다. 블랙핑크의 또 다른 세계를 기대해 볼 수 있다. 지수의 솔로 작품이 K팝적이고, 블랙핑크 틀 안에서는 도무지 상상하기 어려운 것이었다. 네 멤버의 네 방향 이정표가 완성됐다. 이를 따라


In [6]:
outputs[0].outputs[0].text

'블랙핑크 멤버들이 각각의 솔로 작품으로 좋은 반응을 얻고 있다. 블랙핑크는 2023년 YG엔터테인먼트와 팀 활동 재계약을 맺고, 개인 활동은 각자 새로운 둥지를 찾거나 꾸려서 하기로 했다. 리사, 로제, 제니도 각각의 작품이 화제를 모았다. 지수는 2월 14일 미니 앨범 ‘아모르타주(AMORTAGE)’를 발매하여 네 멤버 모두 각자의 작업물을 내놓게 됐다. 타이틀곡 ‘어스퀘이크(earthquake)’는 각 부분이 서로 다른 방식으로 선명하게 인상을 남기며, 호흡을 짚어가는 보컬이 멜로디컬해지고, 1990년대 댄스가요 같은 인상마저 준다. ‘바우트 투 블로(Bout to blow)”라는 가사 그대로 터질 것만 같던 긴장이 시원하게 해소되는 후렴 부분이 있다. ‘이제 노래가 제자리로 돌아왔다’는 기분을 느끼게 하는 안정감 있는 뮤직비디오와 함께, K팝의 정수를 가져와 멋들어지게 표현한 ‘부티’가 흐른다는 느낌을 주는 뮤직비디오가 있다. 지수의 노선은 K팝적이고, 블랙핑크 틀 안에서는 도무지 상상하기 어려운 새로운 블랙핑크의 세계를 기대할 수 있다.'

In [56]:
print(input_ids)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1544,  13806,    220,   2366,     20,    271,   2675,    527,
            264,  11190,  15592,  18328,     13,   5321,   4320,    279,   1217,
            596,   4860,  47626,     13, 113783,  34804, 101003,  67119,  24486,
          15592, 101139,  30426,  25941,  95252,  29726, 119519,     13,  41820,
         110257, 109760,  19954, 112107, 108280, 104834, 102893, 111964,  34983,
          92769,     13, 128009, 128006,    882, 128007,    271, 107837, 123503,
            220,    508, 123590,  78453, 110174,  18359, 120693, 107417, 103170,
         101603, 105204,  20565, 110217, 101738,  18359,  89946,  20565,  35495,
         107138, 123503, 102484,  34804,    220,     20, 117594,  89946,  14705,
            242,  91040, 112521,  24140, 102244, 102484,  34804,  78453, 110174,
          21028,  17196,    

In [53]:
print("Original eos_token_id:", tokenizer.eos_token_id)
print("Original bos_token_id:", tokenizer.bos_token_id)

eos_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")
print("Using eos_id:", eos_id)


Original eos_token_id: 128009
Original bos_token_id: 128000
Using eos_id: 128009


# 2. summarize를 수행하기 위한 파인튜닝

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
from trl import SFTTrainer

In [9]:
# 파인튜닝을 위한 데이터셋 
data = load_dataset("daekeun-ml/naver-news-summarization-ko") #네이버 뉴스 요약 데이터셋 : https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko
data_train = data["train"]
data_valid = data["validation"]
data_test = data["test"]

In [10]:
data_train.shape

(22194, 7)

In [6]:
# 양자화 설정 (GPU 메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [7]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 모델을 LoRA 학습에 맞게 준비
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.39s/it]


In [8]:
# LoRA 설정
peft_config = LoraConfig(
    r=16,                # LoRA의 rank
    lora_alpha=32,       # LoRA alpha 매개변수
    lora_dropout=0.05,   # LoRA 레이어의 드롭아웃 비율
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[     # LoRA를 적용할 모듈
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ]
)
# LoRA 모델 생성
model = get_peft_model(model, peft_config)

In [ ]:

#파인튜닝을 위한 데이터 프롬프트 문 작성 함수.
# def generate_prompts(example):
#     prompt_list = []
#     for i in range(len(example['document'])):
#         prompt_list.append(
# f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>다음 글을 요약해주세요:
# {example['document'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# {example['summary'][i]}<|eot_id|>"""
#         )

In [ ]:
# 데이터 형식 변환 함수 (목적에 맞게 수정 필요)
def format_prompt(sample):
    return f"""
        다음 문장을 요약해주세요.: \n
        {sample["document"]} \n

        요약:
        {sample["summary"]}
        """


dataset = data_train.map(format_prompt)